# Introduction to Ray
---
(*Suggested Time To Complete: 30 minutes*)

Welcome! [detailed description and introduction goes here]

<!-- - Simple Primitives: flexibly compose distributed applications with tasks, actors, and objects in native Python code.
- Multi-cloud: run the same Ray code on any cloud --AWS, GCP, Azure -- or even on-prem.
- Dynamic Scaling: Ray Core can automatically scale (using Ray Autoscaler) up or down to smoothly handle changing compute load.
- Massive Scalability: Ray Core can easily scale to thousands of cores, and is getting more scalable with every release.
- Open Community / Ecosystem: With a vibrant dedicated community and rich ecosystem of integrations, fixes, and best practices are easy to find.
- Laptop -> Cluster With Ease: With Ray Client, going from laptop to cluster is as easy as changing 1 variable -->

**Why Ray?**

[discuss: scaling is a necessity, scaling is hard; make distributed computing easy and simple for everyone]
- Machine Learning is pervasive
- Distributed computing is a necessity
- Python is the default language for DS/ML

- Simple Primitives: flexibly compose distributed applications with tasks, actors, and objects in native Python code.
- Dynamic Scaling: Ray Core can automatically scale (using Ray Autoscaler) up or down to smoothly handle changing compute load.
- Massive Scalability: Ray Core can easily scale to thousands of cores, and is getting more scalable with every release.
- Open Community / Ecosystem: With a vibrant dedicated community and rich ecosystem of integrations, fixes, and best practices are easy to find.
- Laptop -> Cluster With Ease: With Ray Client, going from laptop to cluster is as easy as changing 1 variable
- Multi-cloud: Run the same Ray code on any cloud -- AWS, GCP, Azure -- or even on-premise.

**Prerequisites**

[insert pre-reqs or target audience for this notebook]

**Notebook Outline**

- Introduction to Ray
- Part One: Ray Core
- Part Two: Ray AIR
    - Ray Data
    - Ray Train
    - Ray Tune
    - Ray Serve
    - Ray RLlib
- Part Three: Ray Ecosystem
- Part Four: Ray Clusters
- Homework
- Next Steps

**Learning Goals**

[after this notebook, you will be able to...]

Emmy's Notes:

- Add more detailed description of what Ray is, the motivation behind Ray, what we will cover in this notebook, and the learning goals.
- Add Map of Ray image at the top

# Part One: Ray Core
---
Ray Core is a low-level, distributed computing framework for Python with a concise core API, and you can think of it as the foundation that Ray's data science libraries (Ray AIR) and third-party integrations (Ray Ecosystem) are built on. This simple and general-purpose Python library enables every developer to easily build scalable, distributed systems that run on your laptop, cluster, cloud or Kubernetes.

Ray sets up and manages clusters of computers so that you can run distributed tasks on them. A Ray cluster consists of nodes that are connected to each other via a network. You can program against the so-called driver, the program root, which lives on the head node. The driver can run jobs, that is a collection of tasks, that are run on the nodes in the cluster. Specifically, the individual tasks of a job are run on worker processes on worker nodes.

**Most Popular Use Cases**

- Using Ray for Highly Parallel Tasks: **description, get actual most popular use case feedback from eng**
- Parallel Model Selection: description

**Emmy's TO DOs:**
- Add image below describing Ray Core's relationship to everything else to come

### 💾 Code Example
In this example, we will use Ray Core to implement a distributed version of the classic algorithm quick sort. Quick sort selects an element from the array as a pivot, partitions the array into two sub-arrays (according to whether they are less than or greater than the pivot), then sorts them recursively. We'll go through the mechanics in depth in the code sample, and you can view an animation of quick sort in action below in Figure **#TBD.**

Quick sort is an example of a divide and conquer algorithm, a strategy of solving a large problem by recursively breaking the problem down into smaller sub-problems. By nature, this strategy lends itself well to a parallel implementation because the execution of each operation independently solves smaller instances of the same problem which is then merged into the final solution.

Below, we will take a tour of the sequential implementation before looking at how to distribute it with Ray. At the end, we will compare the code differences between the two to further highlight how concise Ray Core's API is.

**[INSERT GIF OF QUICKSORT, not now though, it would make the version history so beefy]**

#### Sequential Implementation of Quick Sort

In [ ]:
def partition(collection):
    # Use the last element as the first pivot
    pivot = collection.pop()
    greater, lesser = [], []
    for element in collection:
        if element > pivot:
            greater.append(element)
        else:
            lesser.append(element)
    return lesser, pivot, greater

def quick_sort(collection):
    lesser, pivot, greater = partition(collection)
    lesser = quick_sort(lesser)
    greater = quick_sort(greater)
    return lesser + [pivot] + greater

def quick_sort_sequential(collection):
    lesser, pivot, greater = partition(collection)
    lesser = quick_sort_sequential(lesser)
    greater = quick_sort_sequential(greater)
    return lesser + [pivot] + greater

#### Initialize Ray Cluster to Begin Distributed Implementation

In [ ]:
import ray

if ray.is_initialized:
    ray.shutdown()
cluster_info = ray.init(num_cpus=8)
cluster_info.address_info

#### Use the Same Partition and Quick Sort Functions From Above

In [ ]:
def partition(collection):
    # Use the last element as the first pivot
    pivot = collection.pop()
    greater, lesser = [], []
    for element in collection:
        if element > pivot:
            greater.append(element)
        else:
            lesser.append(element)
    return lesser, pivot, greater

def quick_sort(collection):
    lesser, pivot, greater = partition(collection)
    lesser = quick_sort(lesser)
    greater = quick_sort(greater)
    return lesser + [pivot] + greater

#### Add Decorator and Append `.remote()` to Tasks to Parallelize with Ray

In [ ]:
@ray.remote
def quick_sort_distributed(collection):
    lesser, pivot, greater = partition(collection)
    lesser = quick_sort_distributed.remote(lesser)
    greater = quick_sort_distributed.remote(greater)
    return ray.get(lesser) + [pivot] + ray.get(greater)

#### The Difference

Here's a summary of the changes we made to go from sequential to distributed.

```diff
+ import ray

def partition(collection):
    # Use the last element as the first pivot
    pivot = collection.pop()
    greater, lesser = [], []
    for element in collection:
        if element > pivot:
            greater.append(element)
        else:
            lesser.append(element)
    return lesser, pivot, greater

def quick_sort(collection):
    lesser, pivot, greater = partition(collection)
    lesser = quick_sort(lesser)
    greater = quick_sort(greater)
    return lesser + [pivot] + greater

+ @ray.remote
- def quick_sort_sequential(collection):
def quick_sort_distributed(collection):
    lesser, pivot, greater = partition(collection)
-   lesser = quick_sort_distributed(lesser)
+    lesser = quick_sort_distributed.remote(lesser)
-   greater = quick_sort_sequential(greater)
+    greater = quick_sort_distributed.remote(greater)
-   return less + [pivot] + greater
+    return ray.get(lesser) + [pivot] + ray.get(greater)
```

### Coding Excercise: [this]
Try out this thing

### Summary
1. Introduced to Ray Core and Most Popular Workloads
2. Sequential Implementation of Quicksort
3. Distributed Implementation of Quicksort

#### Key Concepts
#### Key API Elements in This Section
#### Next

# 2. Part Two: Ray AIR
---
Ray AI Runtime (AIR) is an open-source toolkit for building ML applications. It provides libraries for distributed data process, model training, tuning, reinforcement learning, model serving, and more.

## Ray Data
Ray Datasets are the standard way to load and exchange data in Ray libraries and applications. They provide basic distributed data transformations such as map, filter, and repartition, and are compatible with a variety of file formats, data sources, and distributed frameworks.

In [ ]:
# Create a Dataset of Python objects.
ds = ray.data.range(10000)
# -> Dataset(num_blocks=200, num_rows=10000, schema=<class 'int'>)

ds.take(5)
# -> [0, 1, 2, 3, 4]

ds.count()
# -> 10000

# Create a Dataset of Arrow records.
ds = ray.data.from_items([{"col1": i, "col2": str(i)} for i in range(10000)])
# -> Dataset(num_blocks=200, num_rows=10000, schema={col1: int64, col2: string})

ds.show(5)
# -> {'col1': 0, 'col2': '0'}
# -> {'col1': 1, 'col2': '1'}
# -> {'col1': 2, 'col2': '2'}
# -> {'col1': 3, 'col2': '3'}
# -> {'col1': 4, 'col2': '4'}

ds.schema()
# -> col1: int64
# -> col2: string

### Summary
#### Key Concepts
#### Key API Elements in This Section
#### Next

## Ray Train
Ray Train is a lightweight library for distributed deep learning that allows you to easily supercharge your distributed PyTorch and TensorFlow training on Ray.

In [ ]:
import ray.train as train
from ray.train import Trainer
import torch

def train_func():
    # Setup model.
    model = torch.nn.Linear(1, 1)
    model = train.torch.prepare_model(model)
    loss_fn = torch.nn.MSELoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)

    # Setup data.
    input = torch.randn(1000, 1)
    labels = input * 2
    dataset = torch.utils.data.TensorDataset(input, labels)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=32)
    dataloader = train.torch.prepare_data_loader(dataloader)

    # Train.
    for _ in range(5):
        for X, y in dataloader:
            pred = model(X)
            loss = loss_fn(pred, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    return model.state_dict()

trainer = Trainer(backend="torch", num_workers=4)
trainer.start()
results = trainer.run(train_func)
trainer.shutdown()

print(results)

### Summary
#### Key Concepts
#### Key API Elements in This Section
#### Next

## Ray Tune
Ray Tune is a Python library for fast hyperparameter tuning at scale. Easily distribute your trial runs to quickly find the best hyperparameters.

In [ ]:
from ray import tune
 
def objective(step, alpha, beta):
   return (0.1 + alpha * step / 100)**(-1) + beta * 0.1
 
def training_function(config):
   # Hyperparameters
   alpha, beta = config["alpha"], config["beta"]
   for step in range(10):
       # Iterative training function - can be any arbitrary training procedure.
       intermediate_score = objective(step, alpha, beta)
       # Feed the score back back to Tune.
       tune.report(mean_loss=intermediate_score)
 
analysis = tune.run(
   training_function,
   config={
       "alpha": tune.grid_search([0.001, 0.01, 0.1]),
       "beta": tune.choice([1, 2, 3])
   })
 
print("Best config: ", analysis.get_best_config(
   metric="mean_loss", mode="min"))
 
# Get a dataframe for analyzing trial results.
df = analysis.results_df

### Summary
#### Key Concepts
#### Key API Elements in This Section
#### Next

## Ray Serve
Ray Serve lets you serve machine learning models in real-time or batch using a simple Python API. Serve individual models or create composite model pipelines, where you can independently deploy, update, and scale individual components.

In [ ]:
import requests

from sklearn.datasets import load_iris
from sklearn.ensemble import GradientBoostingClassifier

from ray import serve

@serve.deployment(route_prefix="/iris")
class BoostingModel:
    def __init__(self, model):
        self.model = model
        self.label_list = iris_dataset["target_names"].tolist()

    async def __call__(self, request):
        payload = await request.json()
        print(f"Received flask request with data {payload}")

        prediction = self.model.predict([payload["vector"]])[0]
        human_name = self.label_list[prediction]
        return {"result": human_name}

if __name__ == "__main__":

    # Train model.
    iris_dataset = load_iris()
    model = GradientBoostingClassifier()
    model.fit(iris_dataset["data"], iris_dataset["target"])

    # Deploy model
    serve.run(BoostingModel.bind(model))

    # Query model
    sample_request_input = {"vector": [1.2, 1.0, 1.1, 0.9]}
    response = requests.get("http://localhost:8000/iris", json=sample_request_input)
    print(response.text)
    
    # prints
    # Result:
    # {"result": "versicolor"}

### Summary
#### Key Concepts
#### Key API Elements in This Section
#### Next

## Ray RLLib
RLlib is the industry-standard reinforcement larning Python frameowrk built on Ray. Designed for quick interation and a fast path to production, it includes 25+ latest algorithms that are all implemented to run at scale and in multi-agent mode.

In [ ]:
from ray import tune
from ray.rllib.agents.ppo import PPOTrainer
 
tune.run(PPOTrainer, config={
   "env": "CartPole-v0",
   "framework": "torch",
   "log_level": "INFO"
})

### Summary
#### Key Concepts
#### Key API Elements in This Section
#### Next

# Part Three: Ray Ecosystem
---
Up until now, we've covered Ray Core, the low-level, distributed computing framework, as well as Ray AIR, the set of high-level native libraries which include Ray Data, Train, Tune, Serve, and RLlib. In addition, Ray integrates with a growing ecosystem of the most popular Python and machine learning libraries and frameworks that you may already be familiar with.

Instead of trying to create new standards, Ray allows you to scale existing workloads by unifying tools in a common interface. This interface enables you to run tasks in a distributed way, a property most of the respective backends don't have, or not to the same extent.

For example, Ray Datasets is backed by Arrow and comes with many integrations to other frameworks, such as Spark and Dask. Ray Train and RLlib are backed by the full power of Tensorflow and PyTorch. Ray Tune supports algorithms from practically every noteable HPO tool available, including Hyperopt, Optuna, Nevergrad, Ax, SigOpt, and many others. Ray Serve can be used with frameworks such as FastAPI, gradio, and Streamlit. With ample opportunity to extend current projects as well as integrate more backends in the future, a key strength of Ray is this robust design pattern for integration which you can read more about [here](https://www.anyscale.com/blog/ray-distributed-library-patterns).

For a complete list of integrations with links, go to the [Ray Ecosystem](https://docs.ray.io/en/latest/ray-overview/ray-libraries.html#) page in the docs.

### Example: HuggingFace
### Example: XGBoost
### Example: Distributed Scikit-Learn / Joblib

### Summary
#### Key Concepts
#### Key API Elements in This Section
#### Next

# Part Four: Ray Clusters

### Summary
#### Key Concepts
#### Key API Elements in This Section
#### Next

# Homework
---
If you would like to practice your new skills further with some in-depth examples beyond the embedded coding excercises, take a look at this list of suggested problems:
- [Distribute a Classical Algorithm with Ray](https://github.com/ray-project/hackathon5-algo)
    - In this excercise, go to the GitHub repo linked above for details on choosing a classic algorithm implemented in Python, editing the implementation to parallelize the work with Ray, and compare your results against the sequential implementation.


# Next Steps
---
🎉 Congratulations! You have completed your first tutorial on an Introduction to Ray! We discussed the three layers of Ray: Core, AIR, and the Ecosystem. Each library in Ray AIR (Data, Train, Tune, Serve, RLLib) and the ecosystem of integrated libraries runs on Ray Core's distributed execution engine, and with Ray Clusters, you can deploy your workloads on AWS, GCP, Azure, or on Kubernetes.

From here, you can learn and get more involved with our active community of developers and researchers by checking out the following resources:
- 📖 [Ray's "Getting Started" Guides](https://docs.ray.io/en/latest/ray-overview/index.html): A collection of QuickStart Guides for every library including installation walkthrough, examples, blogs, talks, and more!
- 💻 [Official Ray Website](https://www.ray.io/): Browse the ecosystem and use this site as a hub to get the information that you need to get going and building with Ray.
- 💬 [Join the Community on Slack](https://forms.gle/9TSdDYUgxYs8SA9e8): Find friends to discuss your new learnings in our Slack space.
- 📣 [Use the Discussion Board](https://discuss.ray.io/): Ask questions, follow topics, and view announcements on this community forum.
- 🙋‍♀️ [Join a Meetup Group](https://www.meetup.com/Bay-Area-Ray-Meetup/): Tune in on meet-ups to listen to compelling talks, get to know other users, and meet the team behind Ray.
- 🪲 [Open an Issue](https://github.com/ray-project/ray/issues/new/choose): Ray is constantly evolving to improve developer experience. Submit feature requests, bug-reports, and get help via GitHub issues.